# Applying advanced methods with chains

<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter9_notebooks/Ch9_Advanced_Methods_with_Chains.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Purpose of this notebook:**  
Exploring some of the methods that **Langchain** lets' us employ:  
* Asking the LLM general knowledge questions  
* Deriving structured data formats from LLMs responses  
* Setting up a memory feature within the conversation  

**Requirements:**  
* When running in Colab, use this runtime notebook setting: `Python 3, CPU`  
* This code picks OpenAI's API as a choice of LLM, so a paid **API key** is necessary.   

>*```Disclaimer: The content and ideas presented in this notebook are solely those of the authors and do not represent the views or intellectual property of the authors' employers.```*

Install:

In [1]:
# REMARK:
# If the below code error's out due to a Python package discrepency, it may be because new versions are causing it.
# In which case, set "default_installations" to False to revert to the original image:
default_installations = True
if default_installations:
    !pip -q install langchain openai
    !pip -q install -U langchain-openai
else:
    import requests
    text_file_path = "advanced_chains.txt"
    url = "https://raw.githubusercontent.com/python-devops-sre/nlp/master/requirements/" + text_file_path           
    res = requests.get(url)
    with open(text_file_path, "w") as f:
      f.write(res.text)
      
    !pip install -r advanced_chains.txt

Imports:

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import CommaSeparatedListOutputParser
import os
import pandas as pd
import json

Code Settings:

Provide OpenAI's API key:  
Remember, if you'd prefer to use a free LLM, you can do so using examples from the book for replacing a paid LLM with a free LLM.  

In [3]:
os.environ["OPENAI_API_KEY"] = "..."
llm = ChatOpenAI(temperature=0.9, model='gpt-3.5-turbo-instruct')

### Asking the LLM a general knowledge question

In [6]:
simple_question = """Who are the members of Metallica. List them as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template=simple_question))

print(llm_chain.predict())

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: .... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Requesting output structure: Making the LLM provide output in a particular data format

In [ ]:
request_list_format = """List the first 10 elements from the periodical table as comma separated list."""

output_parser = CommaSeparatedListOutputParser()

conversation = LLMChain(
    llm=llm,
    output_parser=output_parser,
    prompt=PromptTemplate.from_template(template=request_list_format, input_variables=[]))

print(conversation.predict())

### Evolve to a fluent conversation: Inserting an element of memory so to have previous interactions as reference and context to follow up prompts

In [ ]:
# request_for_continuous_conversation = """List all the holidays you know as comma separated list.
request_for_continuous_conversation = """
Current conversation:
{history}

Your task:
{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=request_for_continuous_conversation, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory())

# conversation.predict_and_parse(input="From the list, write the first 10 holidays, separate by commas.")
conversation.predict_and_parse(input="Write the first 10 holidays you know, as a comma separated list.")

In [ ]:
conversation.predict_and_parse(input="Observe the list of holidays you printed and remove all the non-religious holidays from the list.")

In [ ]:
advanced_data_structure = """For each of these, tell about the holiday in 2 sentences. Form the output in a json format table. The table's name is "holidays" and the fields are "name" and "description". For each row, the "name" is the holiday's name, and the "description" is the description you generated. The syntax of the output should be a json format, without newline characters.
For instance:
```
{"holidays": [
        {"name": "holiday_name",
         "description": "holiday_description"
        }
        ]}
```
"""

output = conversation.predict(input=advanced_data_structure)
print(output)

In [ ]:
pd.set_option('display.max_colwidth', None)

dict = json.loads(output)
pd.json_normalize(dict["holidays"]).style.set_properties(**{'text-align': 'left'})